# BQML Expense Receipt Data Extractor

*   Uses **BQ Dataframes** and BQML to process - allows notebook runtime to **be very lightweight**

*   **BQML provides at-scale inference** all from within BQ - ***no other compute is required***

*   **A notebook isn't required, this can be done in a query including the persistence**

In [ ]:
import os, json
### DECLARATION ###
'''
  ->DINING EXPENSE RECEIPT EXTRACTOR<-

  INPUT
    This stage reads from input_table for records with category = 'dining_expense_receipt'
    category = JSON_VALUE(generated_result['category'])
    Performs extraction

  OUTPUT
    Writes to output_table
'''

STAGE_NAME = "dining_expense_extraction"
BQ_MODEL_NAME = 'ai-learning-agents.unstructured_data.gemini-2_0-flash'

LOCATION = 'us-central1'

INPUT_DATASET_NAME = 'structured_data_output'
INPUT_TABLE_NAME = 'categorization'
INPUT_FILTER_PREDICATE = f"""AND JSON_VALUE(generated_result['category']) = 'dining_expense_receipt'"""
MAX_RECORDS_PER_RUN = 0 ## 0 for unlimited


OUTPUT_DATASET_NAME = 'structured_data_output'


PROJECT_ID = os.environ['GOOGLE_CLOUD_PROJECT']
input_table = f'{PROJECT_ID}.{INPUT_DATASET_NAME}.{INPUT_TABLE_NAME}'
output_table = f'{PROJECT_ID}.{OUTPUT_DATASET_NAME}.{STAGE_NAME}'

In [ ]:
### PROMPT & CONTROLLED GENERATION SCHEMA ###
BASE_CONTEXT = '''You are an expert reviewer of dining expense receipts.
Your job is to use the provided schema to extract information from the file that is provided.
As you go review the receipt and extract data per the schema, keep in mind how confident you are in each of the values you extract according to the schema.

Here are some specific instructions on formatting of extracted values:
transaction_date should be in YYYY-MM-DD format. For example 2025-01-22.
transaction_time_utc and transaction_time_at_location should be in 24 hour HH:MM:SS format without AM/PM. For example convert "5:23 PM" to 17:23:33
biller_type must be all lower case.
phone numbers should be in the format xxx.xxx.xxxx

There may be cases where you need to infer values based on contextual hints:
for biller_type if you cannot immediately determine a value, scan the file for line items that will hint to the value.

If you cannot extract a value for a field from the file, set the associated value for that field as blank
For the overall_summary_of_extraction field, summarize your confidence of the accurace of the required values you extracted. Provide some overall commentary on what you observed.
'''

RESPONSE_SCHEMA_DICT = {
  "type": "object",
  "properties": {
    "extracted_biller": {
        "type": "object",
        "description": "This object contains information about the entity/company that provided goods or services",
        "properties": {
            "biller_type": {
              "type": "string",
              "description": "This is the business type of the biller. Scan the document for line items to help you decide a value"
            },
            "biller_name": {
              "type": "string",
              "description": "This is the biller name."
            },
            "biller_street_address": {
                "type": "string",
                "description": "This is the biller's street address"
            },
            "biller_city": {
                "type": "string",
                "description": "This is the biller's city"
            },
            "biller_state": {
                "type": "string",
                "description": "This is the biller's state. The values will be 2 character United States state abbreviation if the biller is in the US or 2 character province abbreviation if the biller is in Canada",
                "enum": [
                  "AL", "AK", "AZ", "AR", "CA", "CO", "CT", "DE", "FL", "GA",
                  "HI", "ID", "IL", "IN", "IA", "KS", "KY", "LA", "ME", "MD",
                  "MA", "MI", "MN", "MS", "MO", "MT", "NE", "NV", "NH", "NJ",
                  "NM", "NY", "NC", "ND", "OH", "OK", "OR", "PA", "RI", "SC",
                  "SD", "TN", "TX", "UT", "VT", "VA", "WA", "WV", "WI", "WY"]
            },
            "biller_zip": {
                "type": "string",
                "description": "This is the biller's zip"
            },
            "biller_country_iso3": {
                "type": "string",
                "description": "This is the biller's ISO3 country code - such as USA or CAN etc. Use available location information to infer this"
            },
            "biller_country_iso3__confidence": {
                "type": "number",
                "format": "float",
                "minimum": 0,
                "maximum": 1,
                "description": "Determine your confidence of the accuracy of the value you found for biller_country_iso3 and capture it in this field. Use 1 for 100% confident.",
            },
            "biller_country_iso3__inference_explanation": {
                "type": "string",
                "description": "Explain how you arrived at your value for biller_country_iso3",
            },
            "biller_phone": {
                "type": "string",
                "description": "This is the biller's zip"
            }
        },
        "required": [
            "biller_name",
            "biller_street_address",
            "biller_city",
            "biller_state",
            "biller_zip",
            "biller_country_iso3",
            "biller_phone"
        ]
    },
    "biller_order_identifier": {
        "type": "string",
        "description": "This string contains a unique order identifier or order number. If you can't find a value, leave this blank",
    },
    "transaction_date": {
        "type": "string",
        "description": "This string contains the date of the transaction in YYYY-MM-DD format",
        "example": "convert March 2 2024 to 2024-03-02"
    },
    "transaction_time_at_location": {
        "type": "string",
        "format": "time",
        "description": "This string contains the time of the transaction in HH:MM:SS format",
    },
    "transaction_time_utc": {
        "type": "string",
        "format": "time",
        "description": "This string contains the time of the transaction in HH:MM:SS format in the UTC timezone. Use the biller location to determine the UTC offset",
    },
    "transaction_time_utc__confidence": {
        "type": "number",
        "format": "float",
        "minimum": 0,
        "maximum": 1,
        "description": "Determine your confidence of the accuracy of the value you found for transaction_time_utc and capture it in this field. Use 1 for 100% confident.",
    },
    "transaction_time_utc__inference_explanation": {
        "type": "string",
        "description": "Explain how you arrived at your value for transaction_time_utc.",
    },
    "line_items": {
        "type": "array",
        "description": "This object contains line items of the provided goods or services which list a price. Do not include items without a price",
        "items": {
            "type": "object",
            "properties": {
                "line_item_number": {
                    "type": "number",
                    "description": "This is a sequential enumeration of the line number."
                },
                "name": {
                    "type": "string",
                    "description": "Name of the line item"
                },
                "quantity": {
                  "type": "integer",
                  "description": "The quantity of the line item"
                },
                "subtotal": {
                    "type": "number",
                    "description": "The subtotal for the line item. Default value is 0.00"
                }
            },
            "required": [
                "line_item_number",
                "name",
                "quantity",
                "subtotal"
            ]
        }
    },
    "bill_subtotal": {
        "type": "number",
        "description": "This string contains the subtotal of the line items - the sum of the all of the line item prices",
    },
    "bill_subtotal__confidence": {
        "type": "number",
        "format": "float",
        "minimum": 0,
        "maximum": 1,
        "description": "Determine your confidence of the accuracy of the value you found for bill_subtotal and capture it in this field. Use 1 for 100% confident.",
    },
    "bill_subtotal__inference_explanation": {
        "type": "string",
        "description": "Explain how you arrived at your value for bill_subtotal",
    },
    "bill_tax": {
        "type": "number",
        "description": "This string contains the tax total",
    },
    "bill_tax__confidence": {
        "type": "number",
        "format": "float",
        "minimum": 0,
        "maximum": 1,
        "description": "Determine your confidence of the accuracy of the value you found for bill_tax and capture it in this field. Use 1 for 100% confident.",
    },
    "bill_tax__inference_explanation": {
        "type": "string",
        "description": "Explain how you arrived at your value for bill_tax__confidence",
    },
    "bill_total": {
        "type": "number",
        "description": "This string contains the total of the all the line item prices plus the bill tax",
    },
    "bill_total__confidence": {
        "type": "number",
        "format": "float",
        "minimum": 0,
        "maximum": 1,
        "description": "Determine your confidence of the accuracy of the value you found for bill_total and capture it in this field. Use 1 for lowest confidence, 5 for highest.",
    },
    "bill_total__inference_explanation": {
        "type": "string",
        "description": "Explain how you arrived at your value for bill_total__confidence",
    },
    "currency": {
        "type": "string",
        "description": "This is the currency type used for the transaction. If there are no explicit cues, use the biller location to determine the currency",
    },
    "currency__confidence": {
        "type": "number",
        "format": "float",
        "minimum": 0,
        "maximum": 1,
        "description": "Determine your confidence of the accuracy of the value you found for currency and capture it in this field. Use 1 for lowest confidence, 5 for highest.",
    },
    "currency__inference_explanation": {
        "type": "string",
        "description": "Explain how you arrived at your value for currency__confidence",
    },
    "overall_summary_of_extraction": {
        "type": "string",
        "description": "Explain overall confidence in what you extracted, include required properties which you had difficulty with and explain why",
    }
  },
  "required": [
      "extracted_biller",
      "line_items",
      "biller_order_identifier",
      "transaction_date",
      "transaction_time_at_location",
      "transaction_time_utc",
      "transaction_time_utc__confidence",
      "transaction_time_utc__inference_explanation",
      "bill_subtotal",
      "bill_subtotal__confidence",
      "bill_subtotal__inference_explanation",
      "bill_tax",
      "bill_tax__confidence",
      "bill_tax__inference_explanation",
      "bill_total",
      "bill_total__confidence",
      "bill_total__inference_explanation",
      "currency",
      "currency__confidence",
      "currency__inference_explanation"
  ]
}

### Output Table Definition & Schema Helper
*   Initializing the output table
*   Converting schema dict massaging

In [ ]:
### OUTPUT TABLE DEFINITION ###

OUTPUT_TABLE_DESCRIPTION = f"This table contains results of {STAGE_NAME} stage processing"

OUTPUT_SCHEMA = [
  {
    "name": "uri",
    "type": "STRING",
    "mode": "REQUIRED",
    "description": "This is the uri to the file in GCS"
  },
  {
    "name": "md5_hash",
    "type": "STRING",
    "mode": "REQUIRED",
    "description": "This is the md5_hash of the file in GCS"
  },
  {
    "name": "_pk",
    "type": "STRING",
    "mode": "REQUIRED",
    "description": "This is a unique id for the record. It is the concat of uri and md5_hash"
  },
  {
    "name": "content_type",
    "type": "STRING",
    "mode": "REQUIRED",
    "description": "This is the content type for the file"
  },
  {
    "name": "prompt",
    "type": "STRING",
    "mode": "REQUIRED",
    "description": "This is prompt that was used"
  },
  {
    "name": "generated_result",
    "type": "JSON",
    "mode": "NULLABLE",
    "description": "This is JSON representation of the categorization output"
  },
  {
    "name": "generated_datetime",
    "type": "DATETIME",
    "mode": "NULLABLE",
    "description": "This is datetime that the file was categorized"
  },
  {
    "name": "usage_metadata",
    "type": "JSON",
    "mode": "REQUIRED",
    "description": "This metadata returned from the inference process"
  },
  {
    "name": "generative_model_name",
    "type": "STRING",
    "mode": "REQUIRED",
    "description": "This is the name of the model used for inference"
  }
]

In [ ]:
from google.cloud import bigquery

client = bigquery.Client()

try:
  table = client.get_table(output_table)
  print(f"Found output table {output_table}")
except Exception as e:
  table = bigquery.Table(output_table, schema=OUTPUT_SCHEMA)
  table.description = OUTPUT_TABLE_DESCRIPTION
  client.create_table(table)
  print(f"Created output table {output_table}")

Found output table ai-learning-agents.structured_data_output.dining_expense_extraction


In [ ]:
def bqml_schema_cleanser(schema: dict):
  values_to_cap = ["object","array","integer","boolean","number","string","float","double","int32","int64","email","byte","time"]
  string_representation = json.dumps(schema)

  for value in values_to_cap:
    string_representation = string_representation.replace(f' "{value}"', f' "{value.upper()}"')

  return json.loads(string_representation)

### Infer with BQML
*   Infer
*   Example query is more complex than necessary to capture usage metadata

In [ ]:
import bigframes.pandas as bfpd

bfpd.options.bigquery.project = PROJECT_ID
bfpd.options.bigquery.location = LOCATION

# Remove newlines, causes problems with SQL
condensed_base_context = ' '.join(BASE_CONTEXT.splitlines()).strip()

cleansed_response_schema_dict = bqml_schema_cleanser(RESPONSE_SCHEMA_DICT)
sql_safe_response_schema = json.dumps(cleansed_response_schema_dict).replace("'","\\'")

limit_clause = f"LIMIT {MAX_RECORDS_PER_RUN}" if MAX_RECORDS_PER_RUN else ""

query = f"""
  SELECT
      uri
    , md5_hash
    , uri || '-' || md5_hash AS _pk
    , content_type
    , prompt
    , PARSE_JSON(JSON_VALUE(ml_generate_text_result['candidates'][0]['content']['parts'][0]['text'])) AS generated_result
    , ml_generate_text_result['usage_metadata'] AS usage_metadata
    , JSON_VALUE(ml_generate_text_result['model_version']) AS generative_model_name
    , CURRENT_DATETIME() AS generated_datetime
  FROM
    ML.GENERATE_TEXT(
      MODEL `{BQ_MODEL_NAME}`,
      (
        SELECT
            CONCAT('{condensed_base_context}', ' ', uri) AS prompt
          , uri
          , md5_hash
          , content_type
        FROM `{input_table}`
        WHERE uri || '-' || md5_hash NOT IN (SELECT DISTINCT _pk FROM `{output_table}`)
        {INPUT_FILTER_PREDICATE}
        {limit_clause}
      ),
      STRUCT(
            0.1 AS temperature
          , 4096 AS max_output_tokens
          , 0.95 AS top_p
          , FALSE AS ground_with_google_search
          , '{sql_safe_response_schema}' AS response_schema
      )
    );
"""

# print(query)
df = bfpd.read_gbq(query)
print(f"Processed {len(df)} record(s).")

Processed 200 record(s).


### Write Dataframe

In [ ]:
### WRITE ###
df.to_gbq(
    destination_table=output_table,
    if_exists='append',  # Options: 'replace', 'append', 'fail'
)
print(f"Wrote {len(df)} record(s) to {output_table}")

Wrote 200 record(s) to ai-learning-agents.structured_data_output.dining_expense_extraction
